This is copy of kaggle notebook made by **Shahraiz**
Link to notebook: https://www.kaggle.com/shahraizanwar/age-based-text-classification-russian-corpus

**Prepare colab to work with Kaggle Data Sets.**

In [1]:
! pip install -q kaggle

In [2]:
! pip install kaggle --upgrade

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.9)


In [3]:
from google.colab import files

In [4]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"armankizatov","key":"9ec6bd9c97096b18d00bb9fbb00805a0"}'}

In [5]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets list

ref                                                     title                                              size  lastUpdated          downloadCount  
------------------------------------------------------  -----------------------------------------------  ------  -------------------  -------------  
unanimad/us-election-2020                               US Election 2020                                  429KB  2020-11-15 13:41:13           4546  
manchunhui/us-election-2020-tweets                      US Election 2020 Tweets                           353MB  2020-11-09 18:51:59           2422  
antgoldbloom/covid19-data-from-john-hopkins-university  COVID-19 data from John Hopkins University          2MB  2020-11-16 06:04:17           1648  
headsortails/us-election-2020-presidential-debates      US Election 2020 - Presidential Debates           199MB  2020-10-23 16:56:10            450  
etsc9287/2020-general-election-polls                    Election, COVID, and Demographic Data by Cou

In [8]:
!kaggle datasets download oldaandozerskaya/fiction-corpus-for-agebased-text-classification

 98% 498M/509M [00:07<00:00, 85.3MB/s]
100% 509M/509M [00:07<00:00, 68.9MB/s]


In [10]:
!unzip fiction-corpus-for-agebased-text-classification.zip

Archive:  fiction-corpus-for-agebased-text-classification.zip
replace abstracts/adults_1.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

**Needed dependencies.**

In [12]:
import torch
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
import seaborn as sns

**Extracting description of data set.**

In [13]:
description_file = "/content/description.csv"
columns = ['file_name', 'book_title', 'author', 'age_rating', 'genres']
desc = pd.read_csv(description_file, encoding="utf-8", delimiter=';', names=columns)
desc.head()

,file_name,book_title,author,age_rating,genres
0,children_1.txt,Три четверти,Анна Красильщик,12,Детская проза|Детские книги
1,children_2.txt,Повести и рассказы,Аркадий Гайдар,12,Детская проза|Детские книги
2,children_3.txt,"Оленёнок Крапинка, или Бархатистый носик",Холли Вебб,6,Детская проза|Детские книги
3,children_4.txt,Сара и роль мечты,Холли Вебб,6,Детская проза|Детские книги
4,children_5.txt,Как Гитлер украл розового кролика,Джудит Керр,6,Детская проза|Детские книги


**Loading train&test data from directories**

In [30]:
text_location = "/content/train/adults_1.txt"
corpus = open(text_location, "r", encoding='utf-8').read().lower()
splitted_corpus = re.split('; |, |\*|\n|-|–\xa0| ,', corpus)
splitted_corpus

['линии',
 'рю мураками',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'рю мураками',
 '',
 'линии',
 '',
 '',
 '',
 '',
 '',
 'i',
 '',
 'мукаи',
 '',
 '',
 'около шести вечера мукаи заказал номер в отеле',
 'люкс в районе синдзюку. он надел черные очки и старался держаться иначе',
 'чем дома или на работе',
 'как будто опасался',
 'что его могут узнать. проходя через гостиничный холл',
 'он еще раз убедился',
 'что за ним никто не наблюдает.',
 '',
 'мукаи было тридцать лет',
 'и он работал в «сучиру фото лайбрэри». он не был известным фотографом',
 'а его работа заключалась главным образом в покупке любительских фотографий для последующей перепродажи газетчикам. хозяйкой заведения была дама сорока лет',
 'которую звали акико мошидзуки. она начала заниматься этим бизнесом после смерти мужа',
 'в те времена',
 'когда таких фотоагентств было совсем мало. ей было тогда чуть больше тридцати. под офис она приспособила свою двухкомнатную квартиру',
 'в которой до этого и жила с покойным супруг

In [62]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus if x and len(x) <= 20]
    return tokens

tokenized_corpus = tokenize_corpus(splitted_corpus)
tokenized_corpus

[['линии'],
 ['рю', 'мураками'],
 ['рю', 'мураками'],
 ['линии'],
 ['i'],
 ['мукаи'],
 ['как', 'будто', 'опасался'],
 ['он', 'еще', 'раз', 'убедился'],
 ['в', 'те', 'времена'],
 ['однажды', 'мукаи'],
 ['десять', 'лет', 'назад'],
 ['он', 'был', 'всего'],
 ['белых', 'фотоснимков'],
 ['а', 'именно', '«в', 'этом', 'что'],
 ['я', 'же', 'не', 'говорю'],
 ['гортензий', 'под', 'дождем'],
 ['городских', 'парков'],
 ['но'],
 ['с', 'другой', 'стороны'],
 ['то', 'проблемка…»'],
 ['а', 'вернувшись'],
 ['приобрели', '–', 'в', 'кредит'],
 ['видишь', 'ли'],
 ['и', 'мукаи', 'понял'],
 ['то', 'недосягаемой'],
 ['как', 'он.'],
 ['н'],
 ['да'],
 ['что'],
 ['поэтому'],
 ['маки'],
 ['но'],
 ['выйдя', 'за', 'мукаи'],
 ['иногда', '–', 'бизнесмен'],
 ['персональную', 'линию'],
 ['все'],
 ['считали'],
 ['маки', 'была', 'довольно'],
 ['то'],
 ['что', 'ты', 'рассказываешь'],
 ['довольно', 'печально.'],
 ['заметь'],
 ['чего', 'не', 'было?'],
 ['и', 'мать'],
 ['я', 'подумал'],
 ['н'],
 ['да…', 'ну'],
 ['твою', 'мак

In [63]:
vocabulary = []
for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

In [64]:
word2idx

{'3d': 630,
 'i': 3,
 'ii': 154,
 'iii': 336,
 'iv': 485,
 'v': 617,
 'vi': 692,
 '«а': 544,
 '«ах': 421,
 '«в': 25,
 '«вот': 696,
 '«выпуск»': 621,
 '«высокий»': 496,
 '«да': 340,
 '«номер»': 622,
 '«ну»?': 312,
 '«нью': 325,
 '«пинк': 602,
 '«прости': 465,
 '«така»': 495,
 '«что': 500,
 '«эй!»': 667,
 '«яма»': 497,
 'а': 23,
 'а!': 503,
 'а?': 595,
 'ага': 177,
 'ага.': 394,
 'аденин': 731,
 'азабу»': 479,
 'ай': 605,
 'алло!': 487,
 'анимация': 631,
 'ах': 271,
 'а…': 198,
 'бамбуковая': 459,
 'бас': 532,
 'белый': 648,
 'белых': 21,
 'беспокоит.': 493,
 'бизнесмен': 60,
 'бон[1': 240,
 'бренди.': 411,
 'бросила': 288,
 'будете': 221,
 'будете?': 219,
 'будешь?': 280,
 'будто': 6,
 'буду': 223,
 'бы': 109,
 'бы.': 188,
 'бывали': 571,
 'бывают': 276,
 'был': 19,
 'была': 65,
 'были': 231,
 'было': 111,
 'было!': 569,
 'было?': 72,
 'быть': 419,
 'в': 12,
 'вам': 593,
 'ванной': 166,
 'ведущего': 397,
 'ведь': 426,
 'век': 725,
 'верить…': 300,
 'верно': 677,
 'верно.': 192,
 'верно?

In [65]:
window_size = 2
idx_pairs = []
# for each sentence
for sentence in tokenized_corpus:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

In [66]:
idx_pairs[:10]

array([[1, 2],
       [2, 1],
       [1, 2],
       [2, 1],
       [5, 6],
       [5, 7],
       [6, 5],
       [6, 7],
       [7, 5],
       [7, 6]])

In [67]:
def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x
  
  #Input layer is just the center word encoded in one-hot manner. It dimensions are [1, vocabulary_size]

In [68]:
embedding_dims = 10

class NNModule(nn.Module):
  def __init__(self):
    super(NNModule, self).__init__()
    self.input = nn.Linear(vocabulary_size, embedding_dims)
    self.out = nn.Linear(embedding_dims, vocabulary_size)

  def forward(self, x):
    hidden = self.input(x)
    x = F.relu(hidden)
    x = self.out(x)

    return x

In [87]:
learning_rate = 0.01
model = NNModule()
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [88]:
num_epochs = 500
loss_vals = []
W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)

for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:
        x = Variable(get_input_layer(data)).float()
        y_true = Variable(torch.from_numpy(np.array([target])).long())
        y_pred = model(x)

        loss = criterion(y_pred.view(1, -1), y_true)
        loss_val += loss.data.item()
        loss.backward()
        optim.step()
        optim.zero_grad()
        
    if epo % 1 == 0:    
        print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')

Loss at epo 0: 6.878846380908416
Loss at epo 1: 6.047000765800476
Loss at epo 2: 5.816676064870243
Loss at epo 3: 5.739524347605143
Loss at epo 4: 5.673657229321492
Loss at epo 5: 5.6646900861023814
Loss at epo 6: 5.6743154078852145
Loss at epo 7: 5.66081209472954
Loss at epo 8: 5.635996867547129
Loss at epo 9: 5.617081769900681
Loss at epo 10: 5.621134743977778
Loss at epo 11: 5.609254627546304
Loss at epo 12: 5.603571918416427
Loss at epo 13: 5.5830406961840575
Loss at epo 14: 5.584974434882337
Loss at epo 15: 5.582185415719437
Loss at epo 16: 5.584432172111142
Loss at epo 17: 5.578451773979778
Loss at epo 18: 5.576407390147627
Loss at epo 19: 5.572463414702142
Loss at epo 20: 5.579336774797996
Loss at epo 21: 5.570639083895418
Loss at epo 22: 5.578433670060543
Loss at epo 23: 5.569289549548529
Loss at epo 24: 5.566798124326786
Loss at epo 25: 5.56188822091617
Loss at epo 26: 5.560600600591515
Loss at epo 27: 5.564442052707184
Loss at epo 28: 5.5702132221215
Loss at epo 29: 5.5689421

KeyboardInterrupt: ignored